In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras import Model
from tensorflow.keras.layers import (RandomFlip, RandomRotation, RandomContrast, 
                                    GlobalAveragePooling2D, Dense, BatchNormalization)
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B2
from tensorflow.keras.optimizers import SGD

2024-03-14 13:26:55.742272: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 13:26:55.742293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 13:26:55.742794: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
seed = 42
dataset_path = '/home/deeplearning/AI/datasets/TrashNet'
epochs = 30
batch_size = 32
img_height = 224
img_width = 224

In [4]:
data_augmentation = Sequential([
    RandomFlip(mode="horizontal", seed=seed),
    RandomRotation(factor=0.05, seed=seed),
    RandomContrast(factor=0.2, seed=seed)
])

In [5]:
train_ds_raw = image_dataset_from_directory(dataset_path + '/train', batch_size=batch_size)
train_ds = train_ds_raw.map(lambda img, label: (data_augmentation(img), label)).shuffle(100, seed=seed).prefetch(tf.data.AUTOTUNE)

val_ds = image_dataset_from_directory(dataset_path + '/val', batch_size=batch_size)

Found 1768 files belonging to 6 classes.
Found 328 files belonging to 6 classes.


In [6]:
class_names = train_ds_raw.class_names
n_classes = len(class_names)

In [7]:
def train(lr, weight_decay, max_epochs):
    print(f'lr: {lr:.6f}, weight_decay:{weight_decay:.6f}')
    base_model = EfficientNetV2B2(weights="imagenet", include_top=False)
    avg = GlobalAveragePooling2D()(base_model.output)
    output = Dense(n_classes, activation="softmax")(avg)
    model = Model(inputs=base_model.input, outputs=output)

    for layer in model.layers:
        if isinstance(layer, BatchNormalization):
            layer.momentum = 0.90 # max(1 − 10/s, 0.9), where s is the number of steps per epoch, for this dataset and batch size, s= 56

    optimizer = SGD(learning_rate=lr, momentum=0.9, nesterov=True, weight_decay=weight_decay)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    history = model.fit(train_ds, validation_data=val_ds, epochs=max_epochs)

    return history

In [8]:
max_epochs = 10
min_lr, max_lr = -4, -1
min_wd, max_wd = -6, -3
lr_list = np.logspace(min_lr, max_lr, 7).tolist()
wd_list = [0] + np.logspace(min_wd, max_wd, 7).tolist()

In [9]:
for lr in lr_list:
    for wd in wd_list:
        train(lr, wd, max_epochs)

lr: 0.000100, weight_decay:0.000000
Epoch 1/10


2024-03-14 13:27:04.524900: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1710437230.672582  671268 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


56/56 [==============================] - 22s 76ms/step - loss: 1.8345 - accuracy: 0.1618 - val_loss: 1.7796 - val_accuracy: 0.2348
Epoch 2/10
56/56 [==============================] - 7s 55ms/step - loss: 1.7798 - accuracy: 0.2087 - val_loss: 1.7326 - val_accuracy: 0.2744
Epoch 3/10
56/56 [==============================] - 7s 53ms/step - loss: 1.7379 - accuracy: 0.2794 - val_loss: 1.6884 - val_accuracy: 0.3415
Epoch 4/10
56/56 [==============================] - 7s 56ms/step - loss: 1.6892 - accuracy: 0.3326 - val_loss: 1.6590 - val_accuracy: 0.3628
Epoch 5/10
56/56 [==============================] - 7s 54ms/step - loss: 1.6533 - accuracy: 0.3744 - val_loss: 1.6226 - val_accuracy: 0.4421
Epoch 6/10
56/56 [==============================] - 7s 54ms/step - loss: 1.6125 - accuracy: 0.4259 - val_loss: 1.5844 - val_accuracy: 0.4909
Epoch 7/10
56/56 [==============================] - 7s 55ms/step - loss: 1.5875 - accuracy: 0.4548 - val_loss: 1.5522 - val_accuracy: 0.5427
Epoch 8/10
56/56 [=====

2024-03-14 13:28:34.157158: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_1/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


56/56 [==============================] - 19s 68ms/step - loss: 1.8181 - accuracy: 0.1810 - val_loss: 1.7905 - val_accuracy: 0.2226
Epoch 2/10
56/56 [==============================] - 8s 57ms/step - loss: 1.7717 - accuracy: 0.2432 - val_loss: 1.7480 - val_accuracy: 0.2500
Epoch 3/10
56/56 [==============================] - 8s 58ms/step - loss: 1.7304 - accuracy: 0.2647 - val_loss: 1.7025 - val_accuracy: 0.2866
Epoch 4/10
56/56 [==============================] - 8s 59ms/step - loss: 1.6838 - accuracy: 0.3303 - val_loss: 1.6668 - val_accuracy: 0.3232
Epoch 5/10
56/56 [==============================] - 8s 57ms/step - loss: 1.6439 - accuracy: 0.3739 - val_loss: 1.6253 - val_accuracy: 0.3811
Epoch 6/10
56/56 [==============================] - 7s 56ms/step - loss: 1.6019 - accuracy: 0.4202 - val_loss: 1.5854 - val_accuracy: 0.4207
Epoch 7/10
56/56 [==============================] - 8s 57ms/step - loss: 1.5693 - accuracy: 0.4468 - val_loss: 1.5422 - val_accuracy: 0.4604
Epoch 8/10
56/56 [=====

...

2024-03-14 14:40:02.584274: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_49/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


56/56 [==============================] - 19s 71ms/step - loss: 1.2195 - accuracy: 0.5798 - val_loss: 1.7257 - val_accuracy: 0.3933
Epoch 2/10
56/56 [==============================] - 8s 59ms/step - loss: 0.8631 - accuracy: 0.6906 - val_loss: 0.9931 - val_accuracy: 0.6494
Epoch 3/10
56/56 [==============================] - 8s 60ms/step - loss: 0.7479 - accuracy: 0.7302 - val_loss: 0.7425 - val_accuracy: 0.7317
Epoch 4/10
56/56 [==============================] - 8s 59ms/step - loss: 0.6242 - accuracy: 0.7817 - val_loss: 0.8778 - val_accuracy: 0.7470
Epoch 5/10
56/56 [==============================] - 8s 58ms/step - loss: 0.5535 - accuracy: 0.8037 - val_loss: 0.5905 - val_accuracy: 0.7896
Epoch 6/10
56/56 [==============================] - 8s 58ms/step - loss: 0.4076 - accuracy: 0.8620 - val_loss: 0.4537 - val_accuracy: 0.8476
Epoch 7/10
56/56 [==============================] - 8s 59ms/step - loss: 0.3109 - accuracy: 0.8869 - val_loss: 0.5565 - val_accuracy: 0.8232
Epoch 8/10
56/56 [=====

2024-03-14 14:41:31.610751: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_50/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


56/56 [==============================] - 19s 68ms/step - loss: 1.2010 - accuracy: 0.5860 - val_loss: 2.1000 - val_accuracy: 0.3720
Epoch 2/10
34/56 [=================>............] - ETA: 1s - loss: 1.0669 - accuracy: 0.6156

KeyboardInterrupt: 